In [1]:
import os
import re

from langchain_community.document_loaders.confluence import ConfluenceLoader
from dotenv import load_dotenv
from typing import (Any,List)

In [2]:
load_dotenv()
persist_directory = "./chroma_db/"
space_key = "IBW"
confluence_url = os.getenv("CONFLUENE_URL")
username = os.getenv("CONFLUENCE_USERNAME")
api_key = os.getenv("CONFLUENCE_API_KEY")

In [3]:
from langchain_community.document_loaders.confluence import ContentFormat

## 1. Extract the documents
loader = ConfluenceLoader(
    #content_format=ContentFormat.STORAGE,
    url=confluence_url,
    username = username,
    api_key= api_key
)

documents = loader.load(
    #space_key=space_key,
    page_ids= [25397420, 25398513], # [26706543, 185500205], # 398098434 # bankens = 25398483, 25398513
    include_attachments=False,
    include_comments=False,
    limit=100,
    max_pages=1000,
    keep_markdown_format=True,
    )

Received runtime arguments {'page_ids': [25397420, 25398513], 'include_attachments': False, 'include_comments': False, 'limit': 100, 'max_pages': 1000, 'keep_markdown_format': True}. Passing runtime args to `load` is deprecated. Please pass arguments during initialization instead.


In [4]:
from splitters_confluence import ConfluenceMarkdownChunker
from langchain_text_splitters.markdown import MarkdownTextSplitter, MarkdownHeaderTextSplitter

chunk_size = 1000
header_splitter = MarkdownHeaderTextSplitter(
    headers_to_split_on=[("#", "level_1"), ("##", "level_2"), ('###', "level_3")]
)
text_splitter = MarkdownTextSplitter(chunk_size=1000, chunk_overlap=0)

chunker = ConfluenceMarkdownChunker(markdown_header_splitter=header_splitter, markdown_text_splitter=text_splitter)

In [5]:
final = chunker.chunker(documents)

In [10]:
for d in final:
    print(":"*10)
    print(len(d.page_content), d.page_content[:100])
    print(d.metadata["source"])
    print(d.metadata["sub-title"])


::::::::::
644 **Allmän beskrivning** ---
ICA Banken erbjuder två typer av bankkort för privatkunder, ICA Bankkort 
https://ica.atlassian.net/wiki/spaces/IBW/pages/25397420/Bankkort%2C+Bankkort+Plus+och+Migrerat+ICA+kort
**Allmän beskrivning**
::::::::::
511 **Bankkort** ---
* Mastercard Debit
* Går att ha gemensamt.
* Innehåller Reseförsäkring och Matförsä
https://ica.atlassian.net/wiki/spaces/IBW/pages/25397420/Bankkort%2C+Bankkort+Plus+och+Migrerat+ICA+kort
**Bankkort**
::::::::::
974 **Bankkort Plus** ---
* Mastercard, betal och kreditkort
* Går att ha gemensamt.
* Går att delbetala
https://ica.atlassian.net/wiki/spaces/IBW/pages/25397420/Bankkort%2C+Bankkort+Plus+och+Migrerat+ICA+kort
**Bankkort Plus**
::::::::::
109 **Bankkort Plus** **\***För mer information se rutin: Kontantinsättning, kontantuttag och avrundning
https://ica.atlassian.net/wiki/spaces/IBW/pages/25397420/Bankkort%2C+Bankkort+Plus+och+Migrerat+ICA+kort
**Bankkort Plus**
::::::::::
1017 **Migrerat ICA Kort** ---
Mi